**Import thư viện**

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from PIL import Image


ModuleNotFoundError: No module named 'numpy'

**Tải và chuẩn bị dữ liệu**

In [ ]:

# Đường dẫn đến thư mục data
data_dir = Path("./data/")

# Lấy danh sách tất cả các file trong thư mục data
files = list(data_dir.glob("*"))

# Lọc ra chỉ các file có định dạng .png
images = [file for file in files if file.suffix == ".png"]

# Sắp xếp danh sách ảnh
images = sorted(list(map(str, images)))

# Tạo nhãn từ tên file (bỏ phần path và phần mở rộng)
labels = [
    img.split(os.path.sep)[-1].split(".")[0] for img in images
]

# Tìm các ký tự độc nhất trong tất cả các nhãn
characters = set(char for label in labels for char in label)
characters = sorted(list(characters))

print("Số lượng ảnh: ", len(images))
print("Số lượng nhãn: ", len(labels))
print("Số lượng ký tự độc nhất: ", len(characters))
print("Các ký tự: ", characters)
print("Độ dài tối đa của CAPTCHA: ", max([len(label) for label in labels]))


**Vệ sinh dữ liệu**

In [ ]:
# Kiểm tra và hiển thị dữ liệu
plt.figure(figsize=(15, 5))
for i in range(min(5, len(images))):
    img = Image.open(images[i])
    plt.subplot(1, 5, i+1)
    plt.imshow(img)
    plt.title(labels[i])
    plt.axis('off')
plt.show()

# Kiểm tra kích thước ảnh
img = Image.open(images[0])
print(f"Kích thước ảnh thực tế: {img.size}")

**Cấu hình tham số**

In [ ]:
# Kích thước batch cho huấn luyện và kiểm định
batch_size = 16

# Kích thước mong muốn của ảnh
img_width = 160
img_height = 60

# Hệ số downsampling cho các khối tích chập
downsample_factor = 4

# Độ dài tối đa của bất kỳ CAPTCHA nào trong tập dữ liệu
max_length = max([len(label) for label in labels])

**Tạo ánh xạ ký tự <-> số nguyên**

In [ ]:
# Ánh xạ ký tự thành số nguyên
char_to_num = layers.StringLookup(
    vocabulary=list(characters), mask_token=None
)

# Ánh xạ số nguyên trở lại ký tự ban đầu
num_to_char = layers.StringLookup(
    vocabulary=char_to_num.get_vocabulary(), mask_token=None, invert=True
)

**Phân chia dữ liệu và tiền xử lý**

In [ ]:
def split_data(images, labels, train_size=0.9, shuffle=True):
    # 1. Lấy kích thước tổng của tập dữ liệu
    size = len(images)
    # 2. Tạo mảng chỉ số và xáo trộn nó
    indices = np.arange(size)
    if shuffle:
        np.random.shuffle(indices)
    # 3. Lấy kích thước của tập huấn luyện
    train_samples = int(size * train_size)
    # 4. Chia dữ liệu thành các tập huấn luyện và kiểm định
    x_train, y_train = images[indices[:train_samples]], labels[indices[:train_samples]]
    x_valid, y_valid = images[indices[train_samples:]], labels[indices[train_samples:]]
    return x_train, x_valid, y_train, y_valid

# Phân chia dữ liệu thành các tập huấn luyện và kiểm định
x_train, x_valid, y_train, y_valid = split_data(np.array(images), np.array(labels))

def encode_single_sample(img_path, label):
    # 1. Đọc ảnh
    img = tf.io.read_file(img_path)
    # 2. Giải mã và chuyển đổi sang thang độ xám
    img = tf.io.decode_png(img, channels=1)
    # 3. Chuyển đổi sang float32 trong khoảng [0, 1]
    img = tf.image.convert_image_dtype(img, tf.float32)
    # 4. Thay đổi kích thước thành kích thước mong muốn
    img = tf.image.resize(img, [img_height, img_width])
    # 5. Chuyển vị ảnh vì chúng ta muốn chiều thời gian tương ứng với chiều rộng của ảnh
    img = tf.transpose(img, perm=[1, 0, 2])
    # 6. Ánh xạ các ký tự trong nhãn thành số
    label = char_to_num(tf.strings.unicode_split(label, input_encoding="UTF-8"))
    # 7. Trả về một từ điển vì mô hình của chúng ta đang mong đợi hai đầu vào
    return {"image": img, "label": label}



**Chuẩn bị tập dữ liệu**

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = (
    train_dataset.map(
        encode_single_sample, num_parallel_calls=tf.data.AUTOTUNE
    )
    .batch(batch_size)
    .prefetch(buffer_size=tf.data.AUTOTUNE)
)

validation_dataset = tf.data.Dataset.from_tensor_slices((x_valid, y_valid))
validation_dataset = (
    validation_dataset.map(
        encode_single_sample, num_parallel_calls=tf.data.AUTOTUNE
    )
    .batch(batch_size)
    .prefetch(buffer_size=tf.data.AUTOTUNE)
)

**Xây dựng mô hình**

In [ ]:
def build_model():
    # Đầu vào cho ảnh CAPTCHA
    input_img = layers.Input(
        shape=(img_width, img_height, 1), name="image", dtype="float32"
    )
    # Đầu vào cho nhãn
    labels = layers.Input(name="label", shape=(None,), dtype="float32")

    # Xây dựng CNN
    x = layers.Conv2D(32, (3, 3), activation="relu", padding="same", name="Conv1")(input_img)
    x = layers.MaxPooling2D((2, 2), name="pool1")(x)
    x = layers.Conv2D(64, (3, 3), activation="relu", padding="same", name="Conv2")(x)
    x = layers.MaxPooling2D((2, 2), name="pool2")(x)

    # Chúng ta sẽ sử dụng các đặc trưng này cho RNN sequence-to-sequence
    new_shape = ((img_width // 4), (img_height // 4) * 64)
    x = layers.Reshape(target_shape=new_shape, name="reshape")(x)
    x = layers.Dense(64, activation="relu", name="dense1")(x)
    x = layers.Dropout(0.2)(x)

    # RNN
    x = layers.Bidirectional(
        layers.LSTM(128, return_sequences=True, dropout=0.25)
    )(x)
    x = layers.Bidirectional(
        layers.LSTM(64, return_sequences=True, dropout=0.25)
    )(x)

    # Layer đầu ra
    x = layers.Dense(len(char_to_num.get_vocabulary()) + 1, activation="softmax", name="dense2")(x)

    # Thêm layer CTC
    output = layers.Lambda(lambda x: x, name="ctc_loss")(x)

    # Xác định mô hình
    model = keras.models.Model(
        inputs=[input_img, labels], outputs=output, name="ocr_model_v1"
    )

    # Hàm mất mát CTC từ Keras
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001))

    # Thêm hàm mất mát
    def ctc_loss_function(y_true, y_pred):
        batch_len = tf.cast(tf.shape(y_true)[0], dtype="int64")
        input_length = tf.cast(tf.shape(y_pred)[1], dtype="int64")
        label_length = tf.cast(tf.shape(y_true)[1], dtype="int64")

        input_length = input_length * tf.ones(shape=(batch_len, 1), dtype="int64")
        label_length = label_length * tf.ones(shape=(batch_len, 1), dtype="int64")

        loss = keras.backend.ctc_batch_cost(y_true, y_pred, input_length, label_length)
        return loss

    model.add_loss(ctc_loss_function(labels, output))
    
    return model

# Thêm sau hàm build_model() hiện tại

def build_enhanced_model():
    """Phiên bản nâng cao của mô hình với thêm lớp và BatchNormalization"""
    # Đầu vào cho ảnh CAPTCHA
    input_img = layers.Input(
        shape=(img_width, img_height, 1), name="image", dtype="float32"
    )
    # Đầu vào cho nhãn
    labels = layers.Input(name="label", shape=(None,), dtype="float32")

    # Xây dựng CNN sâu hơn
    x = layers.Conv2D(32, (3, 3), activation="relu", padding="same")(input_img)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D((2, 2))(x)
    
    x = layers.Conv2D(64, (3, 3), activation="relu", padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D((2, 2))(x)
    
    x = layers.Conv2D(128, (3, 3), activation="relu", padding="same")(x)
    x = layers.BatchNormalization()(x)

    # Reshape và Dense
    new_shape = ((img_width // 4), (img_height // 4) * 128)
    x = layers.Reshape(target_shape=new_shape)(x)
    x = layers.Dense(128, activation="relu")(x)
    x = layers.Dropout(0.25)(x)

    # RNN với attention
    x = layers.Bidirectional(layers.LSTM(128, return_sequences=True, dropout=0.25))(x)
    x = layers.Bidirectional(layers.LSTM(64, return_sequences=True, dropout=0.25))(x)

    # Layer đầu ra
    x = layers.Dense(len(char_to_num.get_vocabulary()) + 1, activation="softmax")(x)

    # Thêm layer CTC
    output = layers.Lambda(lambda x: x, name="ctc_loss")(x)

    # Xác định mô hình
    model = keras.models.Model(inputs=[input_img, labels], outputs=output, name="ocr_model_enhanced")
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001))

    # Thêm hàm mất mát
    def ctc_loss_function(y_true, y_pred):
        batch_len = tf.cast(tf.shape(y_true)[0], dtype="int64")
        input_length = tf.cast(tf.shape(y_pred)[1], dtype="int64")
        label_length = tf.cast(tf.shape(y_true)[1], dtype="int64")

        input_length = input_length * tf.ones(shape=(batch_len, 1), dtype="int64")
        label_length = label_length * tf.ones(shape=(batch_len, 1), dtype="int64")

        loss = keras.backend.ctc_batch_cost(y_true, y_pred, input_length, label_length)
        return loss

    model.add_loss(ctc_loss_function(labels, output))
    
    return model

# Nếu muốn sử dụng mô hình nâng cao, hãy bỏ comment dòng dưới đây
# model = build_enhanced_model()
# model.summary()

# Xây dựng mô hình
model = build_model()
model.summary()

**Huấn luyện mô hình**

In [ ]:
# Thiết lập early stopping
early_stopping = keras.callbacks.EarlyStopping(
    monitor="val_loss", patience=10, restore_best_weights=True
)

# Huấn luyện mô hình
history = model.fit(
    train_dataset,
    validation_data=validation_dataset,
    epochs=100,
    callbacks=[early_stopping],
)

# Vẽ đồ thị quá trình huấn luyện
plt.plot(history.history["loss"], label="loss")
plt.plot(history.history["val_loss"], label="val_loss")
plt.legend()
plt.show()

**Đánh giá và sử dụng mô hình**

In [ ]:
# Hàm dự đoán CTC
def decode_batch_predictions(pred):
    input_len = np.ones(pred.shape[0]) * pred.shape[1]
    # Sử dụng greedy search (beam_width=1) hoặc beam search với beam_width > 1
    results = keras.backend.ctc_decode(pred, input_length=input_len, greedy=True)[0][0]
    # Lặp qua các kết quả và chuyển đổi các chỉ số thành các ký tự
    output_text = []
    for res in results:
        res = tf.gather(res, tf.where(tf.not_equal(res, -1)))
        res = tf.strings.reduce_join(num_to_char(res)).numpy().decode("utf-8")
        output_text.append(res)
    return output_text

# Kiểm tra mô hình trên tập kiểm định
for batch in validation_dataset.take(1):
    batch_images = batch["image"]
    batch_labels = batch["label"]
    
    preds = model.predict(batch_images)
    pred_texts = decode_batch_predictions(preds)
    
    orig_texts = []
    for label in batch_labels:
        label = tf.strings.reduce_join(num_to_char(label)).numpy().decode("utf-8")
        orig_texts.append(label)
    
    fig, axs = plt.subplots(4, 4, figsize=(15, 5))
    for i in range(min(16, len(pred_texts))):
        img = batch_images[i, :, :, 0].numpy() * 255.0
        img = img.astype(np.uint8)
        title = f"Prediction: {pred_texts[i]}"
        if pred_texts[i] != orig_texts[i]:
            title += f" [True: {orig_texts[i]}]"
        axs[i // 4, i % 4].imshow(img.T, cmap="gray")
        axs[i // 4, i % 4].set_title(title)
        axs[i // 4, i % 4].axis("off")
plt.show()

**Đánh giá độ chính xác**

In [ ]:
# Thêm sau phần hiển thị kết quả dự đoán

# Đánh giá độ chính xác
def calculate_accuracy(dataset):
    """Tính toán độ chính xác trên một tập dataset"""
    correct_predictions = 0
    total_samples = 0
    
    for batch in dataset:
        batch_images = batch["image"]
        batch_labels = batch["label"]
        
        # Lấy nhãn gốc
        orig_texts = []
        for label in batch_labels:
            label = tf.strings.reduce_join(num_to_char(label)).numpy().decode("utf-8")
            orig_texts.append(label)
        
        # Dự đoán
        preds = model.predict(batch_images, verbose=0)
        pred_texts = decode_batch_predictions(preds)
        
        # So sánh dự đoán với nhãn thực tế
        for i in range(len(pred_texts)):
            if pred_texts[i] == orig_texts[i]:
                correct_predictions += 1
            total_samples += 1
            
    return correct_predictions / total_samples

# Tính độ chính xác trên tập kiểm định
validation_accuracy = calculate_accuracy(validation_dataset)
print(f"Độ chính xác trên tập kiểm định: {validation_accuracy:.2%}")

# Tính Character Error Rate (CER)
def calculate_cer(dataset):
    """Tính toán Character Error Rate"""
    total_edit_distance = 0
    total_chars = 0
    
    for batch in dataset:
        batch_images = batch["image"]
        batch_labels = batch["label"]
        
        # Lấy nhãn gốc
        orig_texts = []
        for label in batch_labels:
            label = tf.strings.reduce_join(num_to_char(label)).numpy().decode("utf-8")
            orig_texts.append(label)
        
        # Dự đoán
        preds = model.predict(batch_images, verbose=0)
        pred_texts = decode_batch_predictions(preds)
        
        # Tính edit distance
        for i in range(len(pred_texts)):
            # Levenshtein distance
            edit_distance = tf.strings.length(orig_texts[i]) - tf.strings.length(
                tf.strings.substr(
                    orig_texts[i], 0, 
                    tf.strings.length(tf.strings.join([pred_texts[i], orig_texts[i]], separator=""))))
            
            total_edit_distance += edit_distance
            total_chars += len(orig_texts[i])
            
    return float(total_edit_distance) / float(total_chars)

cer = calculate_cer(validation_dataset)
print(f"Character Error Rate (CER): {cer:.2%}")

**Lưu mô hình**

In [ ]:
# Lưu mô hình CTC với kiến trúc và trọng số
model.save_weights('captcha_model_weights.h5')

# Lưu kiến trúc mô hình dưới dạng JSON
json_string = model.to_json()
with open('captcha_model_architecture.json', 'w') as f:
    f.write(json_string)

# Lưu ánh xạ ký tự
np.save('char_to_num.npy', char_to_num.get_vocabulary())

**Tạo hàm dự đoán cho một ảnh đơn lẻ**

In [ ]:
# Thêm sau phần "Lưu mô hình"

# Hàm dự đoán cho một ảnh CAPTCHA đơn lẻ
def predict_single_image(model, num_to_char, image_path):
    """Dự đoán CAPTCHA từ một file ảnh"""
    img = tf.io.read_file(image_path)
    img = tf.io.decode_png(img, channels=1)
    img = tf.image.convert_image_dtype(img, tf.float32)
    img = tf.image.resize(img, [img_height, img_width])
    img = tf.transpose(img, perm=[1, 0, 2])
    img = tf.expand_dims(img, axis=0)
    
    # Dự đoán
    preds = model.predict(img, verbose=0)
    input_len = np.ones(preds.shape[0]) * preds.shape[1]
    results = keras.backend.ctc_decode(preds, input_length=input_len, greedy=True)[0][0]
    result = tf.gather(results[0], tf.where(tf.not_equal(results[0], -1)))
    result_str = tf.strings.reduce_join(num_to_char(result)).numpy().decode("utf-8")
    
    return result_str

# Kiểm tra với một ảnh cụ thể
# test_image = "path/to/test/image.png"
# prediction = predict_single_image(model, num_to_char, test_image)
# print(f"CAPTCHA dự đoán: {prediction}")